In [ ]:
import requests
from requests import Response
url = "https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json"
try:
    response:Response = requests.get(url)
    response_json = response.json()
    print(response.text)
except Exception as e:
    print(e)


name 'JSON' is not defined


In [3]:
import requests
from requests import Response
import csv
import pandas
import os
import sqlite3

In [4]:
df = pandas.read_json('TodayUrgentCase.json',orient='records')
display(df)

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
...,...,...
483,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
484,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
485,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
486,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [ ]:
from pyproj import Transformer

# 創建轉換器
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")
twd97_x = 305689.109
twd97_y = 2768207.525
# 轉換 TWD97 到 WGS84
latitude,longitude = transformer.transform(twd97_x, twd97_y)

print(f'TWD97轉WGS84: Latitude: {latitude:.7}, Longitude: {longitude:.7}')


TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518


twd97是台灣的座標系統
WGS84是世界的座標系統
經度 longitude
緯度 latitude
所以台灣的X = longitude, Y = latitude

In [ ]:
from geopy.geocoders import Nominatim

# 初始化 Nominatim 反向地理編碼器
geolocator = Nominatim(user_agent='PROJ')

def get_address_from_coordinates(lat, lon):
    """ 根據經緯度獲取地址或道路名稱 """
    location = geolocator.reverse((lat, lon), language='zh-TW', exactly_one=True)
    if location:
        return location.address
    return "未找到地址"

# 範例：查詢經緯度 (台北市)
latitude = 25.02066
longitude = 121.5518

address = get_address_from_coordinates(latitude, longitude)
print(address)
print(f"座標 ({latitude}, {longitude}) 對應的地址是：{address}")


7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣


In [47]:
address = "7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣"
list_address = [item.strip() for item in address.split(",")]
list_address.reverse()
print(list_address)
dic_address = {} 


['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']


In [23]:
display(type(response_json))
# print(response_json["features"][0]["properties"])
response_json01 = response_json["features"][0]["properties"]
for k,v in response_json01.items():
    print(f"{k}:{v}")

dict

BILL_CODE:10967113535908
URGENT_RCV_DATE:20241017
URGENT_START_DATE:20241116000000
URGENT_END_DATE:20241116220000
URGENT_ADDRESS1:台北市樂業街108巷1至11號
X1:305689.109
Y1:2768179.406
URGENT_ADDRESS2:台北市樂業街116號圍牆及118巷2~8號
X2:305689.109
Y2:2768179.406


In [5]:
    
def download_data():
    conn = sqlite3.connect("TPEroad.db")
    url = 'https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Bill_code TEXT,
                RCVdate TEXT,
                Start_date TEXT,
                End_date TEXT,
                Address1 TEXT,
                X1 TEXT,
                Y1 TEXT,
                Address2 TEXT,
                X2 TEXT,
                Y2 TEXT
            )
            ''')
            print("Table 'records' created or already exists.")
    except Exception as e:
        print(e)

    else:
        with conn:
            cursor = conn.cursor()
            for i in response_json['features']:
                for k in i['properties']:
                    Bill_code:str = i['properties']["BILL_CODE"]
                    RCVdate = i['properties']["URGENT_RCV_DATE"]
                    Start_date = i['properties']["URGENT_START_DATE"]
                    End_date = i['properties']["URGENT_END_DATE"]
                    Address1 = i['properties']["URGENT_ADDRESS1"]
                    X1 = i['properties']["X1"]
                    Y1 = i['properties']["Y1"]
                    Address2 = i['properties']["URGENT_ADDRESS2"]
                    X2 = i['properties']["X2"]
                    Y2 = i['properties']["Y2"]
                    
                    cursor = conn.cursor()
                    sql = '''INSERT OR IGNORE INTO records(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2)
                                values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                            '''
                    cursor.execute(sql,(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1, Address2, X2, Y2))

In [ ]:
# download_data()

conn = sqlite3.connect('TPEroad.db')
cursor = conn.cursor()
sql = """
    SELECT * FROM RECORDS
"""
cursor.execute(sql)
cursor.close()
conn.close()